<a href="https://colab.research.google.com/github/tamucc-gcl/wrkshp_edna_metabarcoding/blob/main/analysis/initialize_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Post-Bioionformatic Data Processing

This notebook will:



---
## Initialize Environment

In [2]:
#Set-up Computing Environment - Python

## Mount Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
os.environ['COLAB'] = 'TRUE'

## Install Linux programs needed
!apt install libfribidi-dev libglpk-dev libharfbuzz-dev pandoc

## Clone GitHub Repo and move into that repo
local_path = '/content/edna_workshop'
repo  = "tamucc-gcl/wrkshp_edna_metabarcoding"
url = f"https://github.com/{repo}.git"
!git clone {url} {local_path}

## Setup R and move to local directory
os.chdir(local_path)
%reload_ext rpy2.ipython

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 libcolamd2
  libcxsparse3 libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas-dev
  libgraphblas6 libgraphite2-dev libharfbuzz-gobject0 libharfbuzz-icu0 libklu1
  libldl2 libmetis5 libmongoose2 librbio2 libsliplu1 libspqr2
  libsuitesparse-dev libsuitesparseconfig5 libumfpack5 pandoc-data
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev libgraphite2-utils
  texlive-latex-recommended texlive-xetex texlive-luatex pandoc-citeproc
  texlive-latex-extra context wkhtmltopdf librsvg2-bin groff ghc nodejs php
  python ruby libjs-mathjax libjs-katex citation-style-language-styles
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3
  libcmark-gfm-exte

In [3]:
#Set-up Computing Environment - R
%%R -i local_path
# Point R at your Drive-backed library
drive_lib <- "/content/drive/MyDrive/edna_libraries"
dir.create(drive_lib, recursive = TRUE, showWarnings = FALSE)
.libPaths(c(drive_lib, .libPaths()))

#Install renv if it isn't already
if (!require("renv", lib.loc = drive_lib, quietly = TRUE, warn.conflicts = FALSE)) {
  install.packages("renv", lib = drive_lib,
                   quietly = TRUE, warn.conflicts = FALSE)
}

#Install required libraries into google drive if they aren't already
#Sys.setenv(RENV_CONFIG_REPOS_OVERRIDE = "https://cloud.r-project.org")
#Sys.setenv(RENV_CONFIG_REPOS_OVERRIDE = "https://cloud.r-project.org@https://bioconductor.org/packages/3.19/bioc")
renv::restore(library = drive_lib,
              lockfile = paste0(local_path, '/renv.lock'),
              prompt = FALSE)

#Create a persistent storage temporary file directory
persistent_directory <- "/content/drive/MyDrive/edna_libraries/intermediate_files"
dir.create(persistent_directory, recursive = TRUE, showWarnings = FALSE)

The following package(s) will be updated:

# Bioconductor ---------------------------------------------------------------
- GenomeInfoDbData        [* -> 1.2.13]

# Bioconductor 3.20 ----------------------------------------------------------
- BiocGenerics            [* -> 0.52.0]
- BiocVersion             [* -> 3.20.0]
- Biostrings              [* -> 2.74.1]
- GenomeInfoDb            [* -> 1.42.3]
- IRanges                 [* -> 2.40.1]
- S4Vectors               [* -> 0.44.0]
- UCSC.utils              [* -> 1.2.0]
- XVector                 [* -> 0.46.0]
- zlibbioc                [* -> 1.52.0]

# CRAN -----------------------------------------------------------------------
- ape                     [* -> 5.8-1]
- askpass                 [* -> 1.2.1]
- assertthat              [* -> 0.2.1]
- backports               [* -> 1.5.0]
- base64enc               [* -> 0.1-3]
- bit                     [* -> 4.6.0]
- bit64                   [* -> 4.6.0-1]
- blob                    [* -> 1.2.4]
- bro

In addition: There were 11 warnings (use warnings() to see them)


---
## Parameter Setting

In [4]:
%%R
min_read_count <- 1000 #Minimum number of reads in a sample to keep it

---
## Read in Data

In [ ]:
%%R
samples_with_barcodes <- read_tsv(str_c(here::here(),
               'data/rb_input_barcode.txt',
               sep = '/'),
               show_col_types = FALSE) %>%
  select(sample) %>%
  separate_wider_delim(cols = sample,
                       names = c("pool", "sample_id"),
                       delim = "_")

sampling_metadata <- read_tsv(str_c(here::here(),
                                    'data/Simons_MasterSheet_2017-11-10_2.tsv',
                                   sep = '/'),
                             show_col_types = FALSE, na = c('NA', '?', '')) %>%
  clean_names() %>%
  mutate(sample_id = str_replace_all(vial_label, '-', ''),
         .keep = 'unused') %>%
  inner_join(samples_with_barcodes,
             by = 'sample_id')


#there are two samples with sequencing (in abundance matrix/samples_with_barcodes) without any info in the metadata: 1949E & 1751A

abundance_matrix <- read_csv(str_c(here::here(),
                                   'output/rainbow_bridge/zotu_table.csv',
                                   sep = '/'),
                             show_col_types = FALSE)

zotu_sequences <- Biostrings::readDNAStringSet(str_c(here::here(),
                                   'output/rainbow_bridge/zotu_sequences.fasta',
                                   sep = '/'))

zotu_taxonomy <- abundance_matrix %>%
  select(zotu, domain:species, taxid_rank) %>%
  distinct()


---
## Data Summary

In [ ]:
%%R
font_size <- 2
plot_nzotu_nsamples <- joined_zotu_data %>%
  summarise(n_samples = n_distinct(sample_id),
            n_zotus = n_distinct(zotu),
            n_reads = sum(n_reads),
            .by = c(predator_species_name)) %>%
  ggplot(aes(x = n_samples,
             y= n_zotus)) +
  geom_point() +
  geom_text_repel(aes(label = predator_species_name),
                  size = font_size,
                  max.overlaps = 30,
                  fontface="italic") +
  scale_x_continuous(transform = scales::log10_trans()) +
  scale_y_continuous(transform = scales::log10_trans()) +
  annotation_logticks() +
  labs(x = "No. of Samples (log<sub>10</sub>-scale)",
       y = "No. of zOTUs (log<sub>10</sub>-scale)",
       subtitle = "Number of zOTUs as a function of number of samples") +
  theme_classic(base_size = 10) +
  theme(axis.title = element_markdown(),
        panel.background = element_rect(colour = 'black'))

plot_nreads_nsamples <- joined_zotu_data %>%
  summarise(n_samples = n_distinct(sample_id),
            n_zotus = n_distinct(zotu),
            n_reads = sum(n_reads),
            .by = c(predator_species_name)) %>%
  ggplot(aes(x = n_samples,
             y= n_reads)) +
  geom_point() +
  geom_text_repel(aes(label = predator_species_name),
                  size = font_size,
                  max.overlaps = 30,
                  fontface="italic") +
  scale_x_continuous(transform = scales::log10_trans()) +
  scale_y_continuous(transform = scales::log10_trans()) +
  annotation_logticks() +
  labs(x = "No. of Samples (log<sub>10</sub>-scale)",
       y = "No. of Reads (log<sub>10</sub>-scale)",
       subtitle = "Number of reads as a function of number of samples") +
  theme_classic(base_size = 10) +
  theme(axis.title = element_markdown(),
        panel.background = element_rect(colour = 'black'))

plot_nzotus_nreads <- joined_zotu_data %>%
  summarise(n_samples = n_distinct(sample_id),
            n_zotus = n_distinct(zotu),
            n_reads = sum(n_reads),
            .by = c(predator_species_name)) %>%
  ggplot(aes(x = n_reads,
             y= n_zotus)) +
  geom_point() +
  geom_text_repel(aes(label = predator_species_name),
                  size = font_size,
                  max.overlaps = 30,
                  fontface="italic") +
  scale_x_continuous(transform = scales::log10_trans()) +
  scale_y_continuous(transform = scales::log10_trans()) +
  annotation_logticks() +
  labs(x = "No. of Reads (log<sub>10</sub>-scale)",
       y = "No. of zOTUs (log<sub>10</sub>-scale)",
       subtitle = "Number of zOTUs as a function of number of reads") +
  theme_classic(base_size = 10) +
  theme(axis.title = element_markdown(),
        panel.background = element_rect(colour = 'black'))

plot_nzotu_nsamples + plot_nzotus_nreads + plot_nreads_nsamples + plot_spacer() +
  plot_layout(nrow = 2, guides = 'collect', axes = 'collect')

---
## Data Cleaning
Remove samples/zotus which don't meet prespecified criteria:


1.   zOTU sequence must be 313 bp length
2.   Remove zOTUs identified as the predator species
3. Remove Predator species with less than 1,000 reads across all samples/zOTUs

In [ ]:
%%R
msgs <- c()

filter_predators <- function(data, min_reads){
  out <- data %>%
    filter(sum(n_reads) >= min_read_count,
           .by = c(predator_species_name))

  removed_preds <- anti_join(data, out,
                             by = colnames(data)) %>%
    pull(predator_species_name) %>% n_distinct()

  msgs <<- c(msgs, paste0('\n', 'Number of predator species with fewer than ',
          scales::comma(min_reads), ' total reads: ', scales::comma(removed_preds)))
  msgs <<- c(msgs, paste0('Predator species remaining: ', scales::comma(n_distinct(out$predator_species_name)), '\n'))
  out
}

filter_self <- function(data){
  out <- data %>%
    filter(species != predator_species_name)

  removed_zotus <- anti_join(data, out,
                             by = colnames(data)) %>%
    pull(zotu) %>% n_distinct()

  msgs <<- c(msgs, paste0('\n', 'Number of zOTUs matching predator species: ', scales::comma(removed_zotus)))
  msgs <<- c(msgs, paste0('zOTUs remaining: ', scales::comma(n_distinct(out$zotu)), '\n'))

  out
}

# Identify zotus with a non-standard length sequence
zotus_to_keep <- names(zotu_sequences)[Biostrings::width(zotu_sequences) == 313] %>%
  str_extract('Zotu[0-9]+')

msgs <- c(msgs, paste0('\n', 'Number of zOTUs not 313 bp long: ',
                       scales::comma(length(zotu_sequences) - length(zotus_to_keep))))
msgs <- c(msgs, paste0('zOTUs remaining: ', scales::comma(length(zotus_to_keep)), '\n'))

# Main processing
predator_gut_contents <- joined_zotu_data %>%
  filter(zotu %in% zotus_to_keep) %>%
  filter_self() %>%
  select(-species) %>%
  filter_predators(min_read_count)

# Finally, print all collected messages
cat(paste(msgs, collapse = "\n"))

---
## Post Cleaning Data Summary

In [ ]:
%%R
font_size <- 2
plot_nzotu_nsamples <- predator_gut_contents %>%
  summarise(n_samples = n_distinct(sample_id),
            n_zotus = n_distinct(zotu),
            n_reads = sum(n_reads),
            .by = c(predator_species_name)) %>%
  ggplot(aes(x = n_samples,
             y= n_zotus)) +
  geom_point() +
  geom_text_repel(aes(label = predator_species_name),
                  size = font_size,
                  max.overlaps = 30,
                  fontface="italic") +
  scale_x_continuous(transform = scales::log10_trans()) +
  scale_y_continuous(transform = scales::log10_trans()) +
  annotation_logticks() +
  labs(x = "No. of Samples (log<sub>10</sub>-scale)",
       y = "No. of zOTUs (log<sub>10</sub>-scale)",
       subtitle = "Number of zOTUs as a function of number of samples") +
  theme_classic(base_size = 10) +
  theme(axis.title = element_markdown(),
        panel.background = element_rect(colour = 'black'))

plot_nreads_nsamples <- predator_gut_contents %>%
  summarise(n_samples = n_distinct(sample_id),
            n_zotus = n_distinct(zotu),
            n_reads = sum(n_reads),
            .by = c(predator_species_name)) %>%
  ggplot(aes(x = n_samples,
             y= n_reads)) +
  geom_point() +
  geom_text_repel(aes(label = predator_species_name),
                  size = font_size,
                  max.overlaps = 30,
                  fontface="italic") +
  scale_x_continuous(transform = scales::log10_trans()) +
  scale_y_continuous(transform = scales::log10_trans()) +
  annotation_logticks() +
  labs(x = "No. of Samples (log<sub>10</sub>-scale)",
       y = "No. of Reads (log<sub>10</sub>-scale)",
       subtitle = "Number of reads as a function of number of samples") +
  theme_classic(base_size = 10) +
  theme(axis.title = element_markdown(),
        panel.background = element_rect(colour = 'black'))

plot_nzotus_nreads <- predator_gut_contents %>%
  summarise(n_samples = n_distinct(sample_id),
            n_zotus = n_distinct(zotu),
            n_reads = sum(n_reads),
            .by = c(predator_species_name)) %>%
  ggplot(aes(x = n_reads,
             y= n_zotus)) +
  geom_point() +
  geom_text_repel(aes(label = predator_species_name),
                  size = font_size,
                  max.overlaps = 30,
                  fontface="italic") +
  scale_x_continuous(transform = scales::log10_trans()) +
  scale_y_continuous(transform = scales::log10_trans()) +
  annotation_logticks() +
  labs(x = "No. of Reads (log<sub>10</sub>-scale)",
       y = "No. of zOTUs (log<sub>10</sub>-scale)",
       subtitle = "Number of zOTUs as a function of number of reads") +
  theme_classic(base_size = 10) +
  theme(axis.title = element_markdown(),
        panel.background = element_rect(colour = 'black'))

plot_nzotu_nsamples + plot_nzotus_nreads + plot_nreads_nsamples + plot_spacer() +
  plot_layout(nrow = 2, guides = 'collect', axes = 'collect')

---
## Get Predator Information

In [ ]:
%%R
get_fishbase_data <- function(species_df, species_col = 'species',
                              datasets = c('species', 'ecology', 'estimate',
                                           'morphology', 'reproduction', 'morphometrics')){
  #Datasets: the rfishbase functions (quoted) to get information from

  out <- species_df
  for(func_name in datasets){

    func <- get(func_name, envir = asNamespace("rfishbase"))
    search_result <- func(pull(species_df, species_col)) %>%
      janitor::remove_empty(which = 'cols')

    join_vec <- join_by(!!sym(species_col) == 'Species',
                        !!!syms(intersect(colnames(out),
                                          colnames(search_result))))

    out <- out %>%
      left_join(search_result,
                by = join_vec) %>%
      nest(!!sym(str_c('fishbase_search_', func_name)) := str_subset(colnames(search_result), '^Species$', negate = TRUE))
  }

  out
}

if(!file.exists(str_c(here::here(),
                      'intermediate_files/predator_taxonomy.rds',
                      sep = '/'))){
  fishbase_db <- load_taxa() %>%
    clean_names()

  predator_taxonomy <- sampling_metadata %>%
    mutate(predator_species_name = case_when(predator_species_name == "Ophidion welshi" ~ "Ophidion josephi",
                                             TRUE ~ predator_species_name)) %>%
    distinct(predator_species_name) %>%
    pull(predator_species_name) %>%
    tax_name(get = c("kingdom", "phylum", "class", "order", "family", "genus", "species"),
             db = "ncbi") %>%
    as_tibble() %>%
    left_join(fishbase_db %>% select(species, order_fishbase = order), by = "species") %>%
    mutate(order = coalesce(order, order_fishbase),
           .keep = 'unused') %>%
    select(-db, -query) %>%
    get_fishbase_data()
}


---
## Write Cleaned files for future use
TODO - CHANGE TO WRITE TO PERSISTENT STORAGE

In [ ]:
%%R
dir.create(str_c(here::here(), 'intermediate_files', sep = '/'), showWarnings = FALSE)
predator_gut_contents %>%
  pivot_wider(names_from = 'zotu',
              values_from = n_reads,
              values_fill = 0L) %>%
  write_csv(str_c(here::here(),
                  'intermediate_files/filtered_zotu_counts.csv',
                  sep = '/'))

write_csv(zotu_taxonomy,
          str_c(here::here(),
                  'intermediate_files/zotu_taxonomy.csv',
                  sep = '/'))

write_rds(predator_taxonomy,
          str_c(here::here(),
                'intermediate_files/predator_taxonomy.rds',
                sep = '/'))

In [ ]:
#Cleanly unmount Google Drive
drive.flush_and_unmount()